# L.A. trash collection - change by commodity codes

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import datetime as dt

In [3]:
import altair as alt
import altair_latimes as lat
import matplotlib.pyplot as plt

In [4]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Read the collection data

In [13]:
src = pd.read_csv(
    "data/processed/all_data.csv",
    parse_dates=[
        "datetime",
    ],
    dtype={
        "year": "str",
        "quarter": "str",
        "day": "str",
        "month": "str",
        "weekday": "str",
    },
)

In [54]:
df = src[src["year"] != "2021"].copy()

In [55]:
df["month_year"] = pd.to_datetime(df["month_year"], format="%Y-%m")

### Group by commodity codes

In [56]:
ccodes = (
    df.groupby(["year", "commoditycode"])
    .agg({"tons": sum})
    .sort_values(by=["year", "tons"], ascending=False)
    .reset_index()
)

In [57]:
ccodes.head(8)

,year,commoditycode,tons
0,2020,A,889267.50
1,2020,R,259601.52
2,2020,CA,35271.34
3,2020,P,24670.58
4,2020,W,3100.24
5,2020,E,483.73
6,2020,GR,296.80
7,2020,CR,265.51


In [59]:
ccodes_pivot = pd.pivot_table(
    ccodes, columns="year", values="tons", index="commoditycode", aggfunc={"tons": sum}
).reset_index()

In [61]:
before = ["2016", "2017", "2018", "2019"]

In [68]:
ccodes_pivot["before"] = ccodes_pivot[before].mean(axis=1).round(2)

In [69]:
ccodes_pivot["change"] = (
    (ccodes_pivot["2020"] - ccodes_pivot["before"]) / ccodes_pivot["before"] * 100
).round(2)

In [72]:
ccodes_pivot["diff"] = ccodes_pivot["2020"] - ccodes_pivot["before"]

In [73]:
ccodes_pivot

year,commoditycode,2016,2017,2018,2019,2020,before,change,diff
0,A,796118.42,810194.10,814262.48,829133.27,889267.50,812427.07,9.46,76840.43
1,CA,41315.35,46815.93,44149.93,45877.72,35271.34,44539.73,-20.81,-9268.39
2,CR,2305.94,3110.61,2194.45,1368.69,265.51,2244.92,-88.17,-1979.41
3,D,250.56,235.58,194.87,221.83,193.16,225.71,-14.42,-32.55
4,E,860.87,900.56,761.71,735.85,483.73,814.75,-40.63,-331.02
5,GR,NaN,29.57,177.16,283.28,296.80,163.34,81.71,133.46
6,P,21204.94,22348.49,24545.80,24287.73,24670.58,23096.74,6.81,1573.84
7,R,217728.59,213869.67,202186.95,206369.26,259601.52,210038.62,23.60,49562.90
8,W,1139.24,1513.86,1270.66,2015.71,3100.24,1484.87,108.79,1615.37
